In [1]:
# list of supported layers
from tensorflow.keras.layers import (
    Input,
    Activation,
    AveragePooling2D,
    BatchNormalization,
    Conv2D,
    Dense,
    Dropout,
    Flatten,
    GlobalAveragePooling2D,
    GlobalMaxPooling2D,
    Lambda, # only for polynomial activation in the form of `Lambda(lambda x: x**2+x)`
    MaxPooling2D,
    ReLU,
    Softmax,
    )
from tensorflow.keras import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
# load MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [3]:
# convert y_train and y_test to one-hot encoding
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [4]:
# reshape X_train and X_test to 4D tensor
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

In [5]:
inputs = Input(shape=(28,28,1))
out = Conv2D(4, 3, use_bias=False)(inputs)
out = BatchNormalization()(out)
out = Lambda(lambda x: x**2+x)(out) # best practice: use polynomial activation instead of ReLU
out = AveragePooling2D()(out) # best practice: use AveragePooling2D instead of MaxPooling2D
out = Conv2D(16, 3, use_bias=False)(out)
out = BatchNormalization()(out)
out = Lambda(lambda x: x**2+x)(out)
out = AveragePooling2D()(out)
out = GlobalAveragePooling2D()(out) # best practice: use GlobalAveragePooling2D instead of Flatten
out = Dense(10, activation=None)(out)
out = Softmax()(out)
model = Model(inputs, out)

In [6]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 4)         36        
                                                                 
 batch_normalization (BatchN  (None, 26, 26, 4)        16        
 ormalization)                                                   
                                                                 
 lambda (Lambda)             (None, 26, 26, 4)         0         
                                                                 
 average_pooling2d (AverageP  (None, 13, 13, 4)        0         
 ooling2D)                                                       
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 16)        576   

In [7]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['acc']
    )

In [8]:
model.fit(X_train, y_train, epochs=15, batch_size=128, validation_data=(X_test, y_test))

Epoch 1/15


2022-12-15 00:58:20.416051: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


469/469 [==============================] - 5s 11ms/step - loss: 1.4933 - acc: 0.5921 - val_loss: 0.9403 - val_acc: 0.7958
Epoch 2/15
469/469 [==============================] - 6s 13ms/step - loss: 0.6094 - acc: 0.8744 - val_loss: 0.4354 - val_acc: 0.9026
Epoch 3/15
469/469 [==============================] - 7s 15ms/step - loss: 0.3549 - acc: 0.9181 - val_loss: 0.3081 - val_acc: 0.9280
Epoch 4/15
469/469 [==============================] - 5s 11ms/step - loss: 0.2660 - acc: 0.9337 - val_loss: 0.2328 - val_acc: 0.9382
Epoch 5/15
469/469 [==============================] - 5s 11ms/step - loss: 0.2244 - acc: 0.9405 - val_loss: 0.2119 - val_acc: 0.9407
Epoch 6/15
469/469 [==============================] - 5s 11ms/step - loss: 0.1993 - acc: 0.9454 - val_loss: 0.2094 - val_acc: 0.9371
Epoch 7/15
469/469 [==============================] - 6s 12ms/step - loss: 0.1809 - acc: 0.9499 - val_loss: 0.1846 - val_acc: 0.9428
Epoch 8/15
469/469 [==============================] - 6s 13ms/step - loss: 0.169

In [9]:
model.save('best_practice.h5')

In [10]:
model2 = Model(model.input, model.layers[-2].output)
model2.layers[-1]

In [11]:
X = X_test[[0]]
y = model2.predict(X)
y

1/1 [==============================] - 0s 45ms/step


array([[-0.3705054 , -2.1375978 ,  0.7712853 , -2.8083014 , -2.8595123 ,
        -0.9700138 , -9.806515  ,  9.82091   , -8.521987  ,  0.55516744]],
      dtype=float32)

In [12]:
for layer in model.layers:
    print(layer.__class__.__name__, layer.get_config())
    try:
        print(layer.get_config()['function'])
    except:
        pass
    print(layer.get_input_shape_at(0),layer.get_output_shape_at(0))
    try:
        print(layer.get_weights()[0].shape)
        print(layer.get_weights()[1].shape)
    except:
        pass

InputLayer {'batch_input_shape': (None, 28, 28, 1), 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'input_1'}
(None, 28, 28, 1) (None, 28, 28, 1)
Conv2D {'name': 'conv2d', 'trainable': True, 'dtype': 'float32', 'filters': 4, 'kernel_size': (3, 3), 'strides': (1, 1), 'padding': 'valid', 'data_format': 'channels_last', 'dilation_rate': (1, 1), 'groups': 1, 'activation': 'linear', 'use_bias': False, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}
(None, 28, 28, 1) (None, 26, 26, 4)
(3, 3, 1, 4)
BatchNormalization {'name': 'batch_normalization', 'trainable': True, 'dtype': 'float32', 'axis': ListWrapper([3]), 'momentum': 0.99, 'epsilon': 0.001, 'center': True, 'scale': True, 'beta_initializer': {'class_name': 'Zeros', 'config': {}}, 'gamma_initializer

In [13]:
import json

In [14]:
with open("best_practice.json", "w") as f:
    json.dump({'X': X.flatten().tolist(), 'y': y.flatten().tolist()}, f)